<a href="https://colab.research.google.com/github/DariusTheGeek/Radiant-Earth-Spot-the-Crop-Challenge/blob/main/Brainiac_Numpy_Extration_for_25_Periods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install libraries
!pip -qq install rasterio tifffile

In [ ]:
# Import libraries
import os
import glob
import shutil
import gc
from joblib import Parallel, delayed
from tqdm import tqdm_notebook
import h5py

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


import rasterio
import tifffile as tiff

%matplotlib inline
pd.set_option('display.max_colwidth', None)

In [ ]:
# Download data with a frequency of 10 days
def date_finder(start_date):
  season_dates = []
  m = str(start_date)[:10]
  s = str(start_date)[:10]
  for i in range(24):
    date = datetime.strptime(s, "%Y-%m-%d")
    s = str(date + timedelta(days = 10))[:10]
    season_dates.append(datetime.strptime(s, "%Y-%m-%d"))
  seasons_dates = [datetime.strptime(m, "%Y-%m-%d")] + season_dates
  seasons_dates = [np.datetime64(x) for x in seasons_dates]
  return list(seasons_dates)

# If day not in a frequency of 10 days, find the nearest date
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [ ]:
%%time
# Unpack data saved in gdrive to colab
shutil.unpack_archive('/content/drive/MyDrive/CompeData/Radiant/Radiant_Data.zip', '/content/radiant')
gc.collect()

CPU times: user 13min 43s, sys: 4min 22s, total: 18min 5s
Wall time: 27min 23s


In [ ]:
# Load files
train = pd.concat([pd.read_csv(f'/content/radiant/train{i}.csv', parse_dates=['datetime']) for i in range(1, 5)]).reset_index(drop = True)
test = pd.concat([pd.read_csv(f'/content/radiant/test{i}.csv', parse_dates=['datetime']) for i in range(1, 5)]).reset_index(drop = True)
train.file_path = train.file_path.apply(lambda x: '/'.join(['/content', 'radiant'] + x.split('/')[2:]))
test.file_path = test.file_path.apply(lambda x: '/'.join(['/content', 'radiant'] + x.split('/')[2:]))
train.datetime, test.datetime = pd.to_datetime(train.datetime.dt.date), pd.to_datetime(test.datetime.dt.date)
train['month'], test['month'] = train.datetime.dt.month, test.datetime.dt.month
train.head()

,tile_id,datetime,satellite_platform,asset,file_path,month
0,2587,NaT,NaN,documentation,/content/radiant/ref_south_africa_crops_competition_v1_train_labels/_common/documentation.pdf,NaN
1,2587,NaT,NaN,field_ids,/content/radiant/ref_south_africa_crops_competition_v1_train_labels/ref_south_africa_crops_competition_v1_train_labels_2587/field_ids.tif,NaN
2,2587,NaT,NaN,field_info_train,/content/radiant/ref_south_africa_crops_competition_v1_train_labels/_common/field_info_train.csv,NaN
3,2587,NaT,NaN,labels,/content/radiant/ref_south_africa_crops_competition_v1_train_labels/ref_south_africa_crops_competition_v1_train_labels_2587/labels.tif,NaN
4,2587,NaT,NaN,raster_values,/content/radiant/ref_south_africa_crops_competition_v1_train_labels/_common/raster_values.json,NaN


In [ ]:
# Unique months
train.month.unique()

array([nan,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [ ]:
# Bands
bands = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B11','B12','CLM']

In [ ]:
# Function to load tile and extract fields data into a numpy array and convert the same to a dataframe
# Train
def process_tile_train(tile):
  tile_df = train[(train.tile_id == tile)].reset_index(drop = True)

  y = np.expand_dims(rasterio.open(tile_df[tile_df.asset == 'labels'].file_path.values[0]).read(1).flatten(), axis = 1)
  fields = np.expand_dims(rasterio.open(tile_df[tile_df.asset == 'field_ids'].file_path.values[0]).read(1).flatten(), axis = 1)

  tile_df = train[(train.tile_id == tile) & (train.satellite_platform == 's2')].reset_index(drop = True)

  unique_dates = list(tile_df.datetime.unique())
  start_date = tile_df.datetime.unique()[0]
  # Assert
  diff = set([str(x)[:10] for x in date_finder(start_date)]) - set([str(x)[:10] for x in unique_dates])
  if len(diff) > 0:
    missing = list(set([str(x)[:10] for x in date_finder(start_date)]) - set(diff))
    for d in diff:
      missing.append(str(nearest(unique_dates, np.datetime64(d)))[:10])
    dates = sorted([np.datetime64(x) for x in missing]) 
  else:
    dates = date_finder(start_date)

  X_tile = np.empty((256 * 256, 0))

  colls = []
  for date, datec in zip(dates, range(25)):
    for band in bands:
      tif_file = tile_df[(tile_df.asset == band) & (tile_df.datetime == date)].file_path.values[0]
      X_tile = np.append(X_tile, (np.expand_dims(rasterio.open(tif_file).read(1).flatten(), axis = 1)), axis = 1)
      colls.append(str(datec) + '_' + band)
  df = pd.DataFrame(X_tile, columns = colls)
  df['y'], df['fields'] = y, fields
  return df

In [ ]:
# Preprocessing the data in chunks to avoid outofmemmory error
# Train
tiles = train.tile_id.unique()
chunks = [tiles[x:x+50] for x in range(0, len(tiles), 50)]
[len(x) for x in chunks], len(chunks)

([50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50,
  50],
 53)

In [ ]:
# Preprocessing the tiles without storing them in memory but saving them as csvs in gdrive
# Train
for i in range(len(chunks)):
  pd.DataFrame(np.vstack(Parallel(n_jobs=-1, verbose=1, backend="multiprocessing")(map(delayed(process_tile_train), [x for x in chunks[i]])))).to_csv(f'/content/drive/MyDrive/CompeData/Radiant/Seasonality/train/train{i}.csv', index = False)
  gc.collect()

In [ ]:
# Function to load tile and extract fields data into a numpy array and convert the same to a dataframe
# Test
def process_tile_test(tile):
  tile_df = test[(test.tile_id == tile)].reset_index(drop = True)

  fields = np.expand_dims(rasterio.open(tile_df[tile_df.asset == 'field_ids'].file_path.values[0]).read(1).flatten(), axis = 1)

  tile_df = test[(test.tile_id == tile) & (test.satellite_platform == 's2')].reset_index(drop = True)

  unique_dates = list(tile_df.datetime.unique())
  start_date = tile_df.datetime.unique()[0]
  # Assert
  diff = set([str(x)[:10] for x in date_finder(start_date)]) - set([str(x)[:10] for x in unique_dates])
  if len(diff) > 0:
    missing = list(set([str(x)[:10] for x in date_finder(start_date)]) - set(diff))
    for d in diff:
      missing.append(str(nearest(unique_dates, np.datetime64(d)))[:10])
    dates = sorted([np.datetime64(x) for x in missing]) 
  else:
    dates = date_finder(start_date)

  X_tile = np.empty((256 * 256, 0))

  colls = []
  for date, datec in zip(dates, range(25)):
    for band in bands:
      tif_file = tile_df[(tile_df.asset == band) & (tile_df.datetime == date)].file_path.values[0]
      X_tile = np.append(X_tile, (np.expand_dims(rasterio.open(tif_file).read(1).flatten(), axis = 1)), axis = 1)
      colls.append(str(datec) + '_' + band)
  df = pd.DataFrame(X_tile, columns = colls)
  df['fields'] = fields
  return df

In [ ]:
# Preprocessing the data in chunks to avoid outofmemmory error
# Train
tiles = test.tile_id.unique()
chunks = [tiles[x:x+50] for x in range(0, len(tiles), 50)]
[len(x) for x in chunks], len(chunks)

In [ ]:
# Preprocessing the tiles without storing them in memory but saving them as csvs in gdrive
# Train
for i in range(len(chunks)):
  pd.DataFrame(np.vstack(Parallel(n_jobs=-1, verbose=1, backend="multiprocessing")(map(delayed(process_tile_test), [x for x in chunks[i]])))).to_csv(f'/content/drive/MyDrive/CompeData/Radiant/Seasonality/test/test{i}.csv', index = False)
  gc.collect()